# PRELIMINARY TRAINING AND MODELING

Jack Sparke | 13660507 | 05/11/2023

# Package and Library Import

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import tree
import numpy as np
import random
from statistics import mean
import time
import re
import pickle
from sklearn.utils import resample
from collections import Counter
from sklearn.model_selection import StratifiedKFold,cross_validate
from pylab import rcParams
from mpl_toolkits.mplot3d import Axes3D
import datetime as dt
from sklearn.preprocessing import label_binarize
import glob
from sklearn.ensemble import RandomForestClassifier
from joblib import Parallel, delayed
from sklearn import metrics
from sklearn.model_selection import cross_val_score, KFold, cross_val_predict
from sklearn.metrics import make_scorer, f1_score, accuracy_score, roc_curve, roc_auc_score, confusion_matrix
#from sklearn.cross_validation import StratifiedKFold,cross_val_score ## old name
from sklearn.model_selection import StratifiedKFold,cross_val_score
#from sklearn.cross_validation import train_test_split  # old name
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.svm import SVC
from sklearn.feature_selection import mutual_info_classif
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn')



The following was copied over from a set of unorganised python scripts compiled in Spypder.

# Function Creation

In [ ]:
def find_missing_values(seq):
    """
    Finds the missing values in a sequence of data.
    
    Parameters:
    seq (list): A list of sequential data.
    
    Returns:
    missing_values (list): A list of missing values in the sequence.
    """
    missing_values = []
    seq = sorted(seq)
    for i in range(seq[0], seq[-1] + 1):
        if i not in seq:
            missing_values.append(i)
    
    return missing_values

def reduce_sequence(missing_values, df):
    """
    Reduces the sequence to ensure the data is incremental. 
    Important for XGBoost models.
    
    Currently hard-coded to suit the project's needs.
    
    Parameters:
    missing_values (list): A list the missing values, found using the find_missing_values function.
    df (DataFrame): The dataframe of which to reduce the class values to be incremental.
    
    Returns:
    df_fixed (DataFrame): A DataFrame with the class sequence incrementalised.
    """
    df_func = df.sort_values(by=2)
    df_func = df_func.reset_index(drop=True)
    index_1 = df_func[df_func[2] == missing_values[0]+1].index[0]
    df_1 = df_func[:index_1]

    for i in range(0,len(missing_values)):
        if i == 0:
                    
            index_missed = df_func[df_func[2] == missing_values[i]+1].index[0]
            df_spliced = df_func[index_missed:]
            df_spliced[2] = df_spliced[2] - 1 
            df_spliced = df_spliced.reset_index(drop=True)
            index_missed = df_spliced[df_spliced[2] == missing_values[i+1]].index[0]
            df_spliced = df_spliced[:index_missed]

        else:
                    
            index_missed = df_func[df_func[2] == missing_values[i]].index[0]
            
            df_spliced2 = df_func[index_missed:]
            df_spliced2[2] = df_spliced2[2] - 2 
    
    df_fixed = pd.concat([df_1,df_spliced, df_spliced2], axis=0)
    df_fixed = df_fixed.reset_index(drop=True)

    
    return df_fixed


def combine_df():
    """
    Imports the data from each feature extraction method and merges it row-wise.
    
    Returns:
    combined_df (DataFrame): A DataFrame with each of the datasets merged row-wise.
    """
        filepath_list = [r"C:\Users\jacks\OneDrive - Bond University\233\AML\ASSIGN 1\FeatureExtraction_Tchebyshev_Mukundan2014_10Samples_perimage (1).dat",
             r"C:\Users\jacks\OneDrive - Bond University\233\AML\ASSIGN 1\FeatureExtraction_BGLBP_10samples_perimage2.dat",
             r"C:\Users\jacks\OneDrive - Bond University\233\AML\ASSIGN 1\FeatureExtraction_CSLBP_10samples_perimage2.dat",
             r"C:\Users\jacks\OneDrive - Bond University\233\AML\ASSIGN 1\FeatureExtraction_CSSILTP_10samples_perimage2.dat",
             r"C:\Users\jacks\OneDrive - Bond University\233\AML\ASSIGN 1\FeatureExtraction_SILTP_10samples_perimage2.dat",
             r"C:\Users\jacks\OneDrive - Bond University\233\AML\ASSIGN 1\FeatureExtraction_SCSLBP_10samples_perimage2.dat",
             r"C:\Users\jacks\OneDrive - Bond University\233\AML\ASSIGN 1\FeatureExtraction_OLBP_10Samples_perimage2.dat"
             ]


    counter =  0
    combined_df = None
    for path in filepath_list:
        df = pd.read_csv(path, header=None)
        
        if counter > 0:
            df = df.iloc[:, 3:]
        
        if combined_df is None:
            combined_df = df
        else:
            combined_df = pd.concat([combined_df, df], axis=1)
        counter += 1
        
    return combined_df


def dfs_in_list():
    """
    Imports each of the data in a list rather than into a merged DataFrame.
    
    Returns:
    df_list (list): A list of each DataFrame.
    """
    filepath_list = [r"C:\Users\jacks\OneDrive - Bond University\233\AML\ASSIGN 1\FeatureExtraction_Tchebyshev_Mukundan2014_10Samples_perimage (1).dat",
         r"C:\Users\jacks\OneDrive - Bond University\233\AML\ASSIGN 1\FeatureExtraction_BGLBP_10samples_perimage2.dat",
         r"C:\Users\jacks\OneDrive - Bond University\233\AML\ASSIGN 1\FeatureExtraction_CSLBP_10samples_perimage2.dat",
         r"C:\Users\jacks\OneDrive - Bond University\233\AML\ASSIGN 1\FeatureExtraction_CSSILTP_10samples_perimage2.dat",
         r"C:\Users\jacks\OneDrive - Bond University\233\AML\ASSIGN 1\FeatureExtraction_SILTP_10samples_perimage2.dat",
         r"C:\Users\jacks\OneDrive - Bond University\233\AML\ASSIGN 1\FeatureExtraction_SCSLBP_10samples_perimage2.dat",
         r"C:\Users\jacks\OneDrive - Bond University\233\AML\ASSIGN 1\FeatureExtraction_OLBP_10Samples_perimage2.dat"
         ]
    df_list = []
    for path in filepath_list:
        df = pd.read_csv(path, header=None)
        df_list.append(df)
        
    return df_list
        
    

def attr_name(df, df_name):
    """
    A function for adding interpretable names to the features of the data.
    Used when doing feature selection and to filter the data.
    
    The result feature names will be in the form: feature_extraction_methodCol_number.
    E.g., SILTP3, OLBP56, etc.
    
    Parameters:
    l (list): A list of the updated feature names.
    """
    df_fts = df.columns[3:]
    l = []
    for j in df_fts:
        h = df_name + str(j)
        l.append(h)
    
    return l
    
    
def importer(xgboost=0, family=0):
    """
    Import the data to be split into X and y.
    
    Parameters:
    xgboost (int): If 1, removes the missing values in the class variable, and incrementalising this data to be 
                    prepared for xgboost modeling. Default is 0.
    family (int): If 1, the y value returns the family columns instead of the species column. Default is 0.
    
    Returns:
    X (DataFrame): A DataFrame of the input data.
    y (Series): A series of the target variable. Depending on the family parameter, either family or species class.
    """

    df_list = dfs_in_list()
    comb_df = combine_df()
    
    if xgboost == 1:
        missing_values = find_missing_values(comb_df[2].values)
        comb_df = reduce_sequence(missing_values, comb_df)
    
    
    method_names = ['tchebyshev', 'bglbp', 'cslbp', 'cssiltp', 'siltp', 'scslbp', 'olbp']
    
    att_names = []
    
    
    for i in range(0, len(df_list)):
        df = df_list[i]
        l = attr_name(df, method_names[i])
        att_names.append(l)
        
    if family==1:
        X = comb_df.drop([0,1,2], axis=1)
        y = comb_df[0].values
    else:
        X = comb_df.drop([0,1,2], axis=1)
        y = comb_df[2].values
    
    att_names_ext = []

    
    for arr in att_names:
        att_names_ext.extend(arr)
    
    X.columns = att_names_ext
    
    return X, y
    
def check_class_probabilities(probs):
    """
    Ensures the probabilities generated from a model adds up to one of class-wise basis.
    If not, this function normalises the data to do so. 
    
    Parameters:
    probs (Array): Array of the probabilties generated from the model. 
    """
    row_sums = np.sum(probs, axis=1) 

    # Find rows where the sum is not close to 1.0 within a tolerance
    invalid_rows = np.where(np.abs(row_sums - 1.0) > 1e-6)[0]

    if len(invalid_rows) > 0:
        print("Invalid probabilities found in rows:", invalid_rows)
        
        # Normalize the probabilities for the invalid rows
        for row_index in invalid_rows:
            probs[row_index] /= row_sums[row_index]

        print("Probabilities have been normalized.")
    else:
        print("All class probabilities are correctly normalized.")
    
        
def cross_validate_new(estimator, X, y, cv=4):
    """
    Created cross validation function as the Sci-Kit functions too an extensive amount of time.
    Uses the StratifiedKFold function from Sci-Kit to ensure the existence of each class in each fold. 
    
    If the model does not predict at least one of each species, the function is stopped immediately. 
    
    Parameters:
    estimator (object): Machine learning model that has the fit, predict and predict_proba methods.
    X (Series): Input data to be used to fit the model.
    y (Array): Class data to be used to fit the model.
    cv (int): Number of folds. Default is 4.
    
    Returns:
    X (DataFrame): The fitted estimator from the last fold. 
    cv_dict (Dictionary): Dictionary of the metrics for each fold.
    """
    # Initialize lists to store the models and their scores
    accuracy_scores = []
    f1_scores = []
    roc_auc_scores = []
    
    if isinstance(X, pd.DataFrame):
        X = X.to_numpy()
    if isinstance(y, pd.Series):
        y = y.to_numpy()
    # Create a StratifiedKFold cross-validation iterator
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=8)

    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Fit the estimator on the training data
        st = time.time()
        model = estimator.fit(X_train, y_train)

        # Make predictions on the test set
        y_pred = model.predict(X_test)

        # Calculate accuracy and F1 score
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')

        # Append the model and its scores to the lists

        accuracy_scores.append(accuracy)
        f1_scores.append(f1)

        # Calculate ROC AUC score for multi-class classification

        # Convert the labels to one-hot encoded format for ROC AUC
        y_prob = model.predict_proba(X_test)
        
        if y_prob.shape[1] != 925: #Break if the model has not predicted at least one of each species class
            break
        
        check_class_probabilities(y_prob)
        roc_auc = roc_auc_score(y_test, y_prob, average='macro', multi_class='ovr')
        
        roc_auc_scores.append(roc_auc)
        et = time.time()
        elapsed = et-st
        print('iteration done', elapsed)
        
    cv_dict = {'acc': accuracy_scores,
            'f1': f1_scores,
            'roc-auc': roc_auc_scores}
    return model, cv_dict

                     
def ensemble_cv(X, y, cv=4):
    """
     Specialised function to complete cross validation on an ensemble model. 
     Returns a dictionary of the accuracy, f1-score, and ROC-AUC score (averaged on a 'one-versus-rest' approach) per fold.
    
    The function breaks if the model does not predict at least one of each class. 
    
    Parameters:
    X (Series): Input data to be used to fit the model.
    y (Array): Class data to be used to fit the model.
    cv (int): Number of folds. Default is 4.
    Returns:
    ens (list): Ensemble of the classifiers organised into a list. 
    cv_dict (Dictionary): Dictionary of the metrics for each fold.
    """
    cv_dict = {'roc-auc': [],
               'f1': [],
               'acc': []}
    if isinstance(X, pd.DataFrame):
        X = X.to_numpy()
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=8)
    
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    
        model = RandomForestClassifier(n_estimators=10, max_depth = 15, random_state = 8)
        N = 6
        ens = []
        for i in range(N):
            v = model.fit(X_train, y_train)
            ens.append(v)
            print('iteration done')
        
        predictions = np.array([clf.predict(X_test) for clf in ens])
        final_predictions, _ = mode(predictions, axis=0)
        final_predictions = final_predictions.T
        final_predictions = final_predictions.reshape(len(final_predictions),)
        
        accuracy = accuracy_score(y_test, final_predictions)
        f1 = f1_score(y_test, final_predictions, average='macro')

        roc_auc_scores = []
        o = {}
        y_test_new = y_test-1
        classes = np.unique(y_test_new)
        for class_label in range(classes[-1]+1):
            o[class_label] = []
                
        for classifier in ens:
            
            probs =  ens[0].predict_proba(X_test)
            if probs.shape[1] != 925:
                break
            
            if len(probs[0]) != len(classes):
                fin = classes[-1]
                num = fin- len(probs[0])
                probs = np.pad(probs, ((0, 0), (0, num+1)), mode='constant', constant_values=0)
        
        #take classes down by one
            
            for class_label in range(0,classes[-1]+1):
                if class_label in y_test_new:
                    # Using the One-versus-rest approach
                    y_binary = (y_test_new == class_label).astype(int)
                    
                    binary_probabilities = probs[:, class_label]
                    
                    roc_auc = roc_auc_score(y_binary, binary_probabilities)
                    roc_auc_scores.append(roc_auc)
                    o[class_label].append(roc_auc)
                    
                else:
                    continue
            print('metrics done')

        means = {}
            
        # Iterate through the keys and their associated values
        for key, values in o.items():
            if values:  # Check if the list is not empty
                mean = sum(values) / len(values)
                means[key] = mean
        
        roc_auc_final = sum(means.values()) / len(means)
        
        cv_dict['roc-auc'].append(roc_auc_final)
        cv_dict['f1'].append(f1)
        cv_dict['acc'].append(accuracy)
        
        print('fold done')
        
        
    return ens, cv_dict

def undersample_importer(df, reduction_percentage):
    """
    An importer function that reduces the dataset by the percentage entered as an argument.
    This function also keeps class proportion consistent, just scaled down.
    
    Parameters:
    df (DataFrame): Input data to be used to fit the model.
    reduction_percentage (Float): The percentage to reeduce the dataset by.
    Returns:
    X (DataFrame): A DataFrame of the input data.
    y (Series): A series of the target labels for the species classes.
    """
    
    df_list = dfs_in_list()
    reduction_percentage = reduction_percentage
    
    original_class_counts = dict(Counter(comb_df[2]))
    reduced_class_counts = {cls: int(count * (1 - reduction_percentage)) for cls, count in original_class_counts.items()}
    
    undersampled_data = []
    for cls in original_class_counts:
        class_data = df[df[2] == cls]
        num_samples_to_keep = reduced_class_counts[cls]
        undersampled_class = resample(class_data, n_samples=num_samples_to_keep, replace=False)
        undersampled_data.append(undersampled_class)
    
    # Combine the undersampled classes
    final_dataset = pd.concat(undersampled_data)
    
    method_names = ['tchebyshev', 'bglbp', 'cslbp', 'cssiltp', 'siltp', 'scslbp', 'olbp']
    
    att_names = []
    
    
    for i in range(0, len(df_list)):
        df = df_list[i]
        l = attr_name(df, method_names[i])
        att_names.append(l)

    X = final_dataset.drop([0,1,2], axis=1)
    y = final_dataset[2].values
    
    att_names_ext = []

    # Extend the combined_data list with the elements from each array
    for arr in att_names:
        att_names_ext.extend(arr)
    
    X.columns = att_names_ext
    
    return X, y
    

# Exploratory Data Analysis

In [ ]:
df_list = dfs_in_list()

for df in df_list:
    print(df.shape)

Certain data gathered from different feature extraction methods exemplify greater dimensions than others.

In [ ]:
df = combine_df()

print(df.shape)

In [ ]:
missing_values = find_missing_values(df[2].values)
print(missing_values)

There is a substantial number of dimensions to this dataset. This may cause significant issues in future models as the working machine has a limited RAM capacity of 16GB.
Feature selection will be required. 

Furthermore, there are missing classes evident in species class. This will need to be adjusted when creating XGBoost models, this model requires the classes to be incremental.

In [ ]:
df[2].hist(bins=len(df[2].unique()) , figsize=(35, 15), color='black')
plt.title('Distribution of Samples by Species', fontsize=50, alpha=0.75)
plt.ylabel('Number of Samples', fontsize=30, alpha=0.75)
plt.xlabel('Species Class Label', fontsize=30, alpha=0.75)
plt.xticks(rotation=0, fontsize=20, alpha=0.75)
plt.yticks(rotation=0, fontsize=20, alpha=0.75)


In [ ]:
print(np.unique(df[2], return_counts=True))

In [ ]:
print(np.unique(df[2], return_counts=True)[1].max())
print(np.unique(df[2], return_counts=True)[1].min())

There is substantial class imbalance present in the species label of the dataset, with the count for the smallest and largest number of samples per species being 11320 and 10 respectively. 

In [ ]:
df[0].hist(bins=len(df[0].unique()) , figsize=(35, 15), color='black')
plt.title('Distribution of Samples by Family', fontsize=50, alpha=0.75)
plt.ylabel('Number of Samples', fontsize=30, alpha=0.75)
plt.xlabel('Family Class Label', fontsize=30, alpha=0.75)
plt.xticks(rotation=0, fontsize=20, alpha=0.75)
plt.yticks(rotation=0, fontsize=20, alpha=0.75)

In [ ]:
print(np.unique(df[0], return_counts=True))

In [ ]:
print(np.unique(df[0], return_counts=True)[1].max())
print(np.unique(df[0], return_counts=True)[1].min())

# Decision Tree

A Decision Tree (DT) will first be trained on the entireity of the merged dataset. No hyperparamters will be tuned it is assumed that the Random Forest (RF) classifier will perform better. 

In [ ]:
X, y = importer()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2, stratify = y)
model = DecisionTreeClassifier(random_state = 8)
model, cv_dict = cross_validate_new(model, X_train, y_train, cv=4)
print(cv_dict)

In [ ]:
print(mean(cv_dict['roc-auc']))
print(mean(cv_dict['f1']))
print(mean(cv_dict['acc']))

# Random Forest

A Random Forest (RF) trained on the complete dataset. Default parameters are used. 

In [ ]:
model = RandomForestClassifier(n_estimators = 50, random_state = 8)
model, cv_dict = cross_validate_new(model, X_train, y_train, cv=4)
print(cv_dict)

In [ ]:
print(mean(cv_dict['roc-auc']))
print(mean(cv_dict['f1']))
print(mean(cv_dict['acc']))

The RF model shows significant improvement in class distinction ability compared to the DT.
This model will continue being the object of comparison for future models.

# Feature Selection

Feature selection will be impertive in respect to substantial size of the combined dataset (293830, 536). 

## RF Per Feature Extraction Method

First, an RF will be trained on the each of the dataset containing different feature extraction methods.
This is done to determine if a data obtained from a specific feature extraction method works far more effectively than others in classifying wood species.
Doing so will also reduce the model's strain on the computer's memory by reducing dimensionality whilst retaining performance

In [ ]:
method_names = ['tchebyshev', 'bglbp', 'cslbp', 'cssiltp', 'siltp', 'scslbp', 'olbp']

for i in range(len(df_list)):
    X = df_list[i].drop(columns = [0,1,2])
    y = df_list[i][2]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2, stratify = y)
    model = RandomForestClassifier(n_estimators = 50, random_state = 8)
    model, cv_dict = cross_validate_new(model, X_train, y_train, cv=4)
    print('#################################################')
    print(method_names[i])
    print(mean(cv_dict['roc-auc']))
    print(mean(cv_dict['f1']))
    print(mean(cv_dict['acc']))
    print('#################################################')

The data obtained throug the OLBP feature extraction method exhibits the best performance, whilst the models fitted on the CSSILTP data produce least effective classifier.

## Feature Selection by Information Gain

Next, feature selection can be done on the merged dataset, whereby each feature's importance in information gain is calculated.
It is assumed that sampling only the most effective features will generate the best model. 

In [ ]:
X, y = importer()

info_gain = mutual_info_classif(X,y)
method_names = ['tchebyshev', 'bglbp', 'cslbp', 'cssiltp', 'siltp', 'scslbp', 'olbp'] 
att_names = []

for i in range(len(df_list)):
    df = df_list[i]
    l = attr_name(df, method_names[i])
    att_names.append(l)
    
att_names_ext = []

# Extend the combined_data list with the elements from each array
for arr in att_names:
    att_names_ext.extend(arr)

info_gain = pd.DataFrame({'feature': att_names_ext, 'gain': info_gain})
info_gain = info_gain.sort_values(by='gain', ascending=False)

mini = info_gain[info_gain['gain'] >= info_gain['gain'].mean()]
avg = mini.iloc[-1]['feature']


ax = sns.barplot(y= "importance", x = "feature", data = info_gain, palette=("mako"))
#ax.set_xticklabels(ax.get_xticklabels(), rotation=45, fontsize=8)
ax.set_xlabel("Feature", fontsize=20)  # Adjust the x-label fontsize as needed
ax.set_ylabel("Importance", fontsize=20)  # Adjust the y-label fontsize as needed
ax.set(xticklabels=[])
ax.axvline(x=avg, color='red', linestyle='--')
# Set y-tick label font size
ax.tick_params(axis='y', labelsize=20)
sns.set_context("poster")
fig = plt.gcf()
fig.set_size_inches(20, 15)

Next, RF models are fitted with data that uses the features of differing levels.
Specifically, performance of models with features that have an information gain value:
- In the 70th percentile of information gain
- Greater than the mean of information gain

In [ ]:
X, y = importer()

info_gain = info_gain[info_gain['importance'] >= 0.7]

X = X[info_gain['feature']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2, stratify = comb_df[2].values)

model = RandomForestClassifier(n_estimators = 50, random_state=8)
model, cv_dict = cross_validate_new(model, X_train, y_train, cv=4)
print(mean(cv_dict['roc-auc']))
print(mean(cv_dict['f1']))
print(mean(cv_dict['acc']))


In [ ]:
X, y = importer()

info_gain = info_gain[info_gain['importance'] >= info_gain['importance'].mean()]

X = X[info_gain['feature']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2, stratify = comb_df[2].values)

model = RandomForestClassifier(n_estimators = 50, random_state=8)
model, cv_dict = cross_validate_new(model, X_train, y_train, cv=4)
print(mean(cv_dict['roc-auc']))
print(mean(cv_dict['f1']))
print(mean(cv_dict['acc']))

The RF model fitted using the features that had an information gain value greater than the mean peforms better than using features in the 70th percentile. Furthermore, it also performs better than the RF fitted with the completely merged dataset. 
Future models will be fitted using data filtered by these features. 

# RF Hyperparameter Tuning

With feature selection completed, next is to determine which machine learning model is most appropriate for the task.
The default hyperparameters (100 estimators, maximum depth trees) will been extensive for this task, and will be pruned in the hyperparameter tuning phase to retain performance while being less memory exhaustive. 
An RF has been used up to this point, and will be further optimised by adjusting for the number of estimators in each model. 

In [ ]:
X, y = importer()
info_gain = pd.read_pickle("info_gain.pkl")
X = X[info_gain['feature']]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2, stratify = comb_df[2].values)

train_roc_auc_list = []
test_roc_auc_list = []
estimator_range = range(1, 56, 5)
for n_estimators in estimator_range:
    rf = RandomForestClassifier(n_estimators = n_estimators)
    rf, cv_dict = cross_validate_new(rf, X_train, y_train, cv=4)
    print('######################################################')
    print(mean(cv_dict['roc-auc']))
    print(mean(cv_dict['f1']))
    print(mean(cv_dict['acc']))
    print('######################################################')
    print('iteration done')

In [ ]:
plt.figure(figsize = (16,6))
plt.plot(list(estimator_range), train_roc_auc_list, color = 'blue', label = 'Training Accuracy per N Estimators')
plt.plot(list(estimator_range),test_roc_auc_list, color = 'red', label ='Training Accuracy per N Estimators')
plt.xticks(list(estimator_range))
#plt.title(dataset + ' Actual vs  Prediction for  Day', fontsize = 20,fontweight = "bold")
plt.xlabel('Date', fontsize = 18,fontweight = "bold")
#plt.ylabel(dataset + ' Close Price ($)', fontsize = 18,fontweight = "bold")
plt.legend()
plt.grid()
plt.show

The ROC-AUC for an RF stagnates after approximately 25 estimators. Using estimators after this point will cause diminishing returns, as the RF will become significantly larger with each estimator.

In [ ]:
train_roc_auc_list = []
test_roc_auc_list = []
depth_range = range(1, 50, 5)
for depth in depth_range:
    rf = RandomForestClassifier(n_estimators = 25, max_depth = depth)
    rf, cv_dict = cross_validate_new(rf, X_train, y_train, cv=4)
    print('######################################################')
    print(mean(cv_dict['roc-auc']))
    print(mean(cv_dict['f1']))
    print(mean(cv_dict['acc']))
    print('######################################################')
    print('iteration done')

In [ ]:
plt.figure(figsize = (16,6))
plt.plot(list(estimator_range), train_roc_auc_list, color = 'blue', label = 'Training Accuracy per N Depth')
plt.plot(list(estimator_range),test_roc_auc_list, color = 'red', label ='Training Accuracy per N Depth')
plt.xticks(list(estimator_range))
plt.xlabel('Date', fontsize = 18,fontweight = "bold")
plt.ylabel('N depth', fontsize = 18,fontweight = "bold")
plt.legend()
plt.grid()
plt.show

The maximum depth of each tree will be kept constant at 40. 

In [ ]:
rf = RandomForestClassifier(n_estimators = 25, max_depth = 40)
rf, cv_dict = cross_validate_new(rf, X_train, y_train, cv=4)

print(mean(cv_dict['roc-auc']))
print(mean(cv_dict['f1']))
print(mean(cv_dict['acc']))

# XGBoost Training 

XGBoost is an extension of Adaboost, of which was immediately assumed to be a weaker classifier in consideration of the high dimensionality and class distribution of the dataset. 
XGBoost requires incremental class data from 0; when making xgboost = 1 in the importer function, class labels have the missing values removed and the sequence incrementalised so the model can be executed.

The XGBoost was fitted to have RF that was consistent with the chosen hyperparameters: number of estimators: 25, max depth of 40.

In [ ]:
X, y = importer(xgboost = 1)
info_gain = pd.read_pickle("info_gain.pkl")

X = X[info_gain['feature']]
y = y-1 ## Further reduce the sequence to begin from 0
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2, stratify = y)

In [ ]:
classes = np.unique(y)

model = xgb.XGBClassifier(n_estimators = 25, max_depth=40, objective='multi:softmax', 
                          num_class=len(classes), n_jobs = 5, ## Allow parallelisation across 5 cores of the CPU.
                          random_state = 8) 

model, cv_dict = cross_validate_new(model, X_train, y_train, cv=4)
print(mean(cv_dict['roc-auc']))
print(mean(cv_dict['f1']))
print(mean(cv_dict['acc']))

There is a significant difference in the metrics across each fold. Due to the hyperparameters being constant across each folds, this large difference across folds suggest that the XGBoost model is sensitive to changes to class distribution in the dataset.
As such, when dealing with unknown data in the future, a model exemplifying such characterisitcs would not be used regardless.

Despite this, the best performing model is not as effective at classifying compared to the RF model.


Literature suggests that a smaller dataset with smaller distribution of classes will be more effective.
Testing this:

In [ ]:
comb_df = combine_df()

info_gain = pd.read_pickle("info_gain.pkl")

missing_values = find_missing_values(comb_df[2].values)
df_fixed = reduce_sequence(missing_values, comb_df)

df_fixed = df_fixed[df_fixed[2] <= 50] # Classifying for only the first 50 classes

X = df_fixed.drop(columns=[0,1,2])
y = df_fixed[2]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2, stratify=y)


In [ ]:
model = xgb.XGBClassifier(n_estimators = 25, max_depth=40, objective='multi:softmax', 
                          num_class=len(classes), n_jobs = 5, ## Allow parallelisation across 5 cores of the CPU.
                          random_state = 8) 

model, cv_dict = cross_validate_new(model, X_train, y_train, cv=4)
print(mean(cv_dict['roc-auc']))
print(mean(cv_dict['f1']))
print(mean(cv_dict['acc']))

Significantly better peformance compared to using the xgboost on the entirety of the dataset.
Now to compare against the RF.

In [ ]:
clf = RandomForestClassifier(n_estimators=25, max_depth=40, random_state=8)
model, cv_dict = cross_validate_new(clf, X_train, y_train, cv=4)
print(mean(cv_dict['roc-auc']))
print(mean(cv_dict['f1']))
print(mean(cv_dict['acc']))

No difference; understandable as the smaller dataset can lead both models in determining similar decision boundaries that produce equal results.

# Support Vector Machine Training

A support vector machine (SVM) model will be conducted on the training data. 

DO NOT RUN BELOW CODE, EXENSIVE AMOUNT OF TIME TO RUN

In [ ]:
#model = SVC(kernel="poly", degree=8, C=0.5, probability=True)# Probability set equal to true to return probabilites
#model.fit(X_train, y_train)

In [ ]:
#preds = model.predict(X_test)
#probs = model.predict_proba(X_test)

The SVM on the complete dataset even with feature selection took an extended amount of time to complete, with obtaining predictrions taking longer than actually training.
To ensure that all models are covered, the data will be undersampled with class distribution maintained. 
An RF with the optimised parameters will then be run on the undersampled data for comparison.

In [ ]:
comb_df = combine_df()

info_gain = pd.read_pickle("info_gain.pkl")

## The following was done so as to ensure the ROC-AUC score can be calculated.
missing_values = find_missing_values(comb_df[2].values)
df_fixed = reduce_sequence(missing_values, comb_df)
X, y = undersample_importer(df_fixed, 0.7) # Reduce the dataset by 70%

X = X[info_gain['feature']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2, stratify=y)


Using the 'poly' kernal:

In [ ]:
model = SVC(kernel="poly", degree=7, probability=True)
model, cv_dict = cross_validate_new(model, X_train, y_train, cv=4)
print(mean(cv_dict['roc-auc']))
print(mean(cv_dict['f1']))
print(mean(cv_dict['acc']))

Using the 'rbf' kernal:

In [ ]:
model = SVC(kernel="rbf", degree=7, probability=True)
model, cv_dict = cross_validate_new(model, X_train, y_train, cv=4)
print(mean(cv_dict['roc-auc']))
print(mean(cv_dict['f1']))
print(mean(cv_dict['acc']))

Using the 'sigmoid' kernal:

In [ ]:
model = SVC(kernel="sigmoid", degree=7, probability=True)
model, cv_dict = cross_validate_new(model, X_train, y_train, cv=4)
print(mean(cv_dict['roc-auc']))
print(mean(cv_dict['f1']))
print(mean(cv_dict['acc']))

The 'poly' kernal demonstrates the best ROC-AUC score, and will be used for the future training. 

In [ ]:
for i in range(6,13)
    model = SVC(kernel="poly", degree=i, probability=True)
    model, cv_dict = cross_validate_new(model, X_train, y_train, cv=4)
    print(mean(cv_dict['roc-auc']))
    print(mean(cv_dict['f1']))
    print(mean(cv_dict['acc']))
    print('###############################################')

A degree of 10 indicates best ROC-AUC, will be used to compared with the RF below on the undersample dataset. 
The model exemplifies a high ROC-AUC score, however, demonstrates a poor F1 and accuracy score. 

In [ ]:
clf = RandomForestClassifier(n_estimators = 25, max_depth = 40, random_state=8)
clf, cv_dict = cross_validate_new(clf, X_train, y_train, cv=4)
print(mean(cv_dict['roc-auc']))
print(mean(cv_dict['f1']))
print(mean(cv_dict['acc']))

RF shows superior performance on the undersampled data, therefore, the SVM will be disregarded.

# RF Ensemble 

Now with the model chosen, next to determine if an ensemble of RFs is a more effective at classifying the species of wood.

In [ ]:
X, y = importer(xgboost=1) # The classes will be incrementalised to ensure the ROC-AUC score can be calculated. 
info_gain = pd.read_pickle("info_gain.pkl")

X = X[info_gain['feature']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2, stratify = y)
final_predictions, ens_dict, cv_dict = ensemble_cv(X_train, y_train)
print(mean(cv_dict['roc-auc']))
print(mean(cv_dict['f1']))
print(mean(cv_dict['acc']))

An ensemble of RF shows better performance compared to a single RF. This is currently the benchmark model for future training.
However, such a model is highly memory exhaustive.

# Cascade of Ensemble

The idea behind this model is based on the assumption that an RF model will be a more effective classifier on a subset of classes. So, if the input data can be effectively classify the input data based on its family, therefore, running the data on a model especially trained on that subset of family's species should be far more accurate.
Incorrectly predicting the family in the first layer is offset by greater classification capabilities in the sucessive layer.

## RF on Ensemble

Prior testing demonstrated that an ensemble of RF are better classifiers compared to a single RF fitted to classify the species label. 
This will be assumed to be consistent for an ensemble of RF fitted to the family labels.

## RF on Subset of Species Data

Next, to determine if an RF will be a more effective classifier on a subset of species labels

In [ ]:
comb_df = combine_df()
family_list = [1,2,3,4,5,6,7,8,9,10]
comb_df = comb_df[comb_df[0].isin(family_list)]

In [ ]:
X = comb_df.drop(columns=[0,1,2])
y = comb_df[2]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2, stratify = y)
model = RandomForestClassifier(n_estimators = 25, max_depth = 8, random_state=8)
model, cv_dict = cross_validate_new(model, X_train, y_train, cv=4)
print(mean(cv_dict['roc-auc']))
print(mean(cv_dict['f1']))
print(mean(cv_dict['acc']))

As expected, the RF model trained on a subset of data is a more effective classifier. 
An ensemble of models can be applied at this layer, however, there would be diminishing returns as the model would substantially large and more computational expensive. We want the initial layer to be as effective as possible, therefore we will make the first layer an ensemble, but the models trained on a subset of families will only be a single RF model.

## Model Creation

A class object will be created to construct this cascade of ensemble. An auxilary function will be created to do stratified cross validation on the model. 

In [ ]:
class wf_ensemble:
    def __init__(self, chunks=6):
        """
        Initialisation function for the class. Upon construction, methods are added to the class that will be 
            filled as training commences.


        A dictionary is intialised that splits the dataset into its families and corresponding species.
        Ensure that the length of the unique classes in the family column is divisible by the input for chunks, 
            as each chunk will have an equal number of family classes. Note that this corresponds to the number 
            of models that will be produced for the species layer. Default is 6.

        Parameters:
        chunks (int): The number of which the families will be split.

        Returns:
        self.speDict (Dictionary): Dictionary of each chunk's families and associated species.
        """
        self.chunks = chunks
        self.fam_ens = None
        self.fam_predictions = None
        self.probs = {}
        self.roc = {}
        self.metrics = {}
        self.preds_roc = {}
        self.spe_preds = None
        comb_df = combine_df()
        
        comb_df = combine_df()
        # Initialize an empty dictionary to store the key-value pairs
        speDict = {}
        
        # Initialize the starting value
        current_value = 1
        size = len(np.unique(comb_df[0]))//chunks
        # Iterate for each of the 6 lists
        for i in range(chunks):
            sub_list = []  # Initialize an empty list for the current sublist
            for _ in range(size):
                sub_list.append(current_value)  # Add the current value to the sublist
                current_value += 1  # Increment the current value by 1
            
            speDict[i] = {}
            speDict[i]['family'] = sub_list
            speDict[i]['species'] = np.unique(comb_df[comb_df[0].isin(sub_list)][2]).tolist()
        
        self.speDict = speDict

    
    def fam_fit(self, X_train, y_train_family, n_classifiers = 5):
        """
        The method to train the ensemble of RFs. The target will be family column instead.
        
        Parameters:
        X_train (Series): Input data to be used to fit the model.
        y_train_family (Array): Class data to be used to fit the model, for this layer specifically needs to be the family class.
        n_classifiers (int): Number of classifiers in the ensemble.
        
        Returns:
        self.fam_ens (list): Ensemble of classifiers trained to determined the family of the input data.
        """
        
        self.fam_ens = []
        for i in range(n_classifiers):
            model = RandomForestClassifier(n_estimators=25, max_depth=40, random_state=8)
            model.fit(X_train, y_train_family)
            self.fam_ens.append(model)
            print('family training iteration done',i)
        
        
    def fam_predict(self, X):
        """
        Generate a predictions from the ensemble using majority vote. 
        
        Parameters:
        X (Series): Input data to be used to fit the model. In training, this would be the testing data for X. 
        
        Returns:
        self.fam_predictions (array): Predictions for the family of the input data.
        """
        predictions = np.zeros((len(X), len(self.fam_ens)), dtype=int)
        # Predict with each classifier
        for i, classifier in enumerate(self.fam_ens):
            predictions[:, i] = classifier.predict(X)
        
        # Perform majority voting to get the final predictions
        self.fam_predictions = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=1, arr=predictions).astype(int)
            
        print('family prediction done')
        
    def spe_fit(self, X_train, y_train_species, y_train_family):
        """
        The method to train a model for each chunk of families, predicting the species of the input data.
        
        Parameters:
        X_train (Series): Input data to be used to fit the model.
        y_train_species (Array): Labels of the species of the input data. 
        y_train_family (Array): Labels of the family of the input data.
       
        Returns:
        self.speDict_ (estimator): Ensemble of classifiers trained to determined the family of the input data.
        """
        for i in range(self.chunks):
                
            d = np.where(np.isin(y_train_family, self.speDict[i]['family']))
            Xv = X_train.iloc[d[0].tolist()]
            
            yv = y_train_species[d[0]]
            
            model_ = RandomForestClassifier(n_estimators=25, max_depth=40, random_state=8)
            model_.fit(Xv, yv)
            
            self.speDict[i]['model'] = model_
            print('specie training iteration done')
        
    
    def spe_predict(self, X):
        """
        Generate predictions from the input data to determine the species. 
        
        Parameters:
        X (Series): Input data to be used to fit the model. For training, this would be the testing data for X.
        
        Returns:
        self.preds_roc (Array): Predictions for the species of the input data. Done to compute the ROC-AUC score, as the predictions need to correspond to their probability. 
        self.spe_preds (Array): Predictions for the species of the input data, being filled into its associated index.
        self.probs (Array): Probabilities for the predicted class. 
        """

        self.spe_preds = np.zeros((len(X)), dtype=int)

        for i in range(self.chunks):
            
            spe_index = np.where(np.isin(self.fam_predictions, self.speDict[i]['family']))
            spe_X = X.iloc[spe_index[0].tolist()]
        
            self.preds_roc[i] = self.speDict[i]['model'].predict(spe_X)

            self.spe_preds[spe_index] = self.preds_roc[i]
            
            self.probs[i] = self.speDict[i]['model'].predict_proba(spe_X)
            
            print('specie prediction iteration done')
    
        print('species prediction done')
        
    def compute_metrics(self, y):
        """
        Compute metrics using the data. Specifically, the accuracy, f1-score, and ROC-AUC score.
        The ROC-AUC is computed manually on one-versus-rest approach.
        
        Parameters:
        y (Array): Species labels corresponding to the X input data. For training, this is the test labels.
        
        Returns:
        self.metrics (Dictionary): A dictionary for the metrics calculated from the model. Each key corresponds to the metric.
        """
        
        acc = accuracy_score(y, self.spe_preds)
        f1 = f1_score(y, self.spe_preds, average='macro')
        
    
        for i in range(6):
            print(i)
            z = self.spe_preds[np.where(np.isin(self.spe_preds, self.speDict[i]['species']))]
            
            o = {}
            for u in self.speDict[i]['species']:
                o[f'{u}'] = []
            
            classes = np.unique(self.speDict[i]['species'])
            
            
            roc_auc_scores = []
            
            for q in range(len(self.speDict[i]['species'])):
                if classes[q] in np.unique(z):
                    y_true = (self.preds_roc[i] == classes[q]).astype(int)  # Convert to binary for the current class
                    y_scores =  self.probs[i][:, q]
                    roc_auc = roc_auc_score(y_true, y_scores)
                    o[f'{classes[q]}'] = roc_auc
                else:
                    o[f'{classes[q]}'] = 0.5
        
            
            
            self.roc[i] = o
            
        total_sum = 0
        total_count = 0
        
    
        for sub_dict in self.roc.values():
            for value in sub_dict.values():
                total_sum += value
                total_count += 1
        
        # Calculate the average
        roc_average = total_sum / total_count
        
        self.metrics['acc'] = acc
        self.metrics['f1'] = f1
        self.metrics['roc-auc'] = roc_average


def cross_validate_wf_ens(info_gain, cv=4):
    """
    Specialised function to complete cross validation on the cascading ensemble model. 
    
    Parameters:
    info_gain (DataFrame): The DataFrame of features to filter the dataset by. 
    cv (int): Number of folds. Default is 4.
    
    Returns:
    wf_ens (class object): The cascading ensemble model constructed as a class object. 
    cv_dict (Dictionary): Dictionary of the metrics for each fold.
    """
    # Initialize lists to store the models and their scores
    X, y_family = importer(family=1)
    X, y_species = importer()
    
    X = X[info_gain['feature']]
    e = X.columns.to_list()
    accuracy_scores = []
    f1_scores = []
    roc_auc_scores = []
    
    if isinstance(X, pd.DataFrame):
        X = X.to_numpy()

    # Create a StratifiedKFold cross-validation iterator
    print('import done')
    skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=8)

    for train_index, test_index in skf.split(X, y_family):
        X_train, X_test = X[train_index], X[test_index]
        y_train_family, y_test_family = y_family[train_index], y_family[test_index]
        y_train_species, y_test_species = y_species[train_index], y_species[test_index]
        
        
        X_train, X_test = pd.DataFrame(X_train), pd.DataFrame(X_test)

        X_train.columns = e
        X_test.columns = e
        
        X_train = X_train[info_gain['feature']]
        
        wf_ens = wf_ensemble(chunks = 6)
    

        wf_ens.fam_fit(X_train, y_train_family, 3)
        wf_ens.fam_predict(X_test)
        
        
        #X_train = pd.DataFrame(X_train)
        wf_ens.spe_fit(X_train, y_train_family, y_train_species)
        
        for i in range(wf_ens.chunks):
            print(wf_ens.speDict[i]['model'].n_classes_)
        
        #X_test = pd.DataFrame(X_test)
        wf_ens.spe_predict(X_test)
        
        
        wf_ens.compute_metrics(X_test, y_test_species)
        wf_ens.metrics

        accuracy_scores.append(wf_ens.metrics['acc'])
        f1_scores.append(wf_ens.metrics['f1'])

        roc_auc_scores.append(wf_ens.metrics['roc-auc'])
        print('fold done')
        
    cv_dict = {'acc': accuracy_scores,
            'f1': f1_scores,
            'roc-auc': roc_auc_scores}
    
        
    return wf_ens, cv_dict

In [ ]:
info_gain = pd.read_pickle("info_gain.pkl")

wf_ens, cv_dict = cross_validate_wf_ens()
print(mean(cv_dict['roc-auc']))
print(mean(cv_dict['f1']))
print(mean(cv_dict['acc']))

The results show that this is the best performing model. No further training is required as the RF have been optimised.
Next, to test the model on the test data.

## Model Training 

In [ ]:
X, y = importer()
info_gain = pd.read_pickle("info_gain.pkl")

X = X[info_gain['feature']] 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=8, test_size=0.2, stratify = y)

wf_ens, cv_dict = cross_validate_wf_ens()
print(mean(cv_dict['roc-auc']))
print(mean(cv_dict['f1']))
print(mean(cv_dict['acc']))

## Testing

In [ ]:
st = time.time()
wf_ens.fam_predict(X)
wf_ens.spe_predict(X)
wf_ens.compute_metrics(y)
ed = time.time()
elapsed_time = ed - st
print(elapsed_time)

In [ ]:
wf_ens.metrics

### Confusion Matrix

Due to the number of classes, the confusion matrix will be filtered down to only display the classes with the lowest associated ROC-AUC values.

In [ ]:
# Initialize a dictionary to store the worst performing keys for each sub-dictionary
worst_keys = {}

# Define the number of worst keys to retrieve
num_worst_keys = 8

# Iterate through the main dictionary
for sub_dict_name, sub_dict in wf_ens.roc.items():
    # Filter out keys with a value of 1
    filtered_dict = {key: value for key, value in sub_dict.items() if value != 1}
    
    # Find the worst performing keys from the filtered dictionary
    worst_keys[sub_dict_name] = sorted(filtered_dict, key=filtered_dict.get)[:num_worst_keys]


combined_list = [item for sublist in worst_keys.values() for item in sublist]
combined_list = [int(item) for item in combined_list]

confusion_matrix_worst = confusion_matrix(y_test, wf_ens.spe_preds, labels=combined_list[:10])


cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix_worst, display_labels = combined_list[:10])
cm_display.plot()
plt.show()

## Misclassified Classes in Bar Graph

The confusion matrix was not effective in displaying the misclassified classes.
Instead, a bar graph will be used. 

In [ ]:
conf_matrix = confusion_matrix(y, wf_ens.spe_preds)

misclassification_percentage = 1 - np.diag(conf_matrix) / np.sum(conf_matrix, axis=1)

threshold = 0.05
filtered_indices = np.where(misclassification_percentage >= threshold)[0]

bar_width = 0.8  
spacing_factor = 1.5  

bar_positions = np.arange(len(filtered_indices)) * spacing_factor

plt.figure(figsize=(10, 6))
plt.bar(bar_positions, misclassification_percentage[filtered_indices] * 100, width=bar_width, align="center")
plt.xticks(bar_positions, filtered_indices, rotation=45, ha="right")  
plt.xlabel("Class")
plt.ylabel("Misclassification Percentage (%)")
plt.title(f"Misclassification Percentage by Class (Threshold >= {threshold * 100}%)")

plt.show()

In [ ]:

misclassification_count = misclassification_percentage[filtered_indices] * np.sum(conf_matrix[filtered_indices], axis=1)

bar_width = 0.8 
spacing_factor = 1.5 

bar_positions = np.arange(len(filtered_indices)) * spacing_factor

plt.figure(figsize=(10, 6))
plt.bar(bar_positions, misclassification_count, width=bar_width, align="center")
plt.xticks(bar_positions, filtered_indices, rotation=45, ha="right") 
plt.xlabel("Class")
plt.ylabel("Misclassification Count")
plt.title(f"Misclassification Count by Class (Threshold >= {threshold * 100}%)")

plt.show()